# MISC


In [ ]:
import openai
import os

messages = [
    {
        "role": "user",
        "content": "hi",
    },
]

model = "gpt-35-turbo"
client = openai.AzureOpenAI(
    api_key=os.environ.get("AZURE_API_KEY"),
    api_version=os.environ.get("API_VERSION"),
    azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
)
response = client.chat.completions.create(
    model=model,
    messages=messages,
)
response.choices[0].message.content

'Hello! How can I assist you today?'

In [70]:
import os
import openai

messages = [
    {
        "role": "user",
        # "content": "MY NAME IS MINSUK. remember this. I will ask again.",
        "content": "What is my name?",
    },
    # {
    #     "role": "user",
    #     # "content": "MY NAME IS MINSUK. remember this. I will ask again.",
    #     "content": "What is my name?",
    # },
]
model = "gpt-35-turbo"
client = openai.AzureOpenAI(
    api_key=os.environ.get("AZURE_API_KEY"),
    api_version=os.environ.get("API_VERSION"),
    azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
)
response = client.chat.completions.create(
    model=model,
    messages=messages,
    # response_format={"type": "json_object"},
)
response = response.choices[0].message.content
print(response)

I apologize, but I am an AI and I do not have access to personal information.


In [ ]:
def apply_llm(
    input_file: str,
    prompt_file: str = "../llm_prompt.txt",
    # num_chunks: int = -1,
    max_tokens: int = 2000,
    model_token_limit: int = 4096,
):
    model = "gpt-3.5-turbo"
    # Initialize the tokenizer
    tokenizer = tiktoken.encoding_for_model(model)

    input = get_string_from_text_file(input_file)
    # prompt = get_string_from_text_file(prompt_file) + input + "```"
    prompt = get_string_from_text_file(prompt_file)
    # Encode the text_data into token integers
    token_integers = tokenizer.encode(input)

    # Split the token integers into chunks based on max_tokens
    # chunk_size = max_tokens - len(token_integers)
    # if num_chunks != -1:
    #     chunk_size = len(token_integers) // num_chunks
    # else:
    #     chunk_size = model_token_limit - 50
    chunks = [
        token_integers[i : i + max_tokens]
        for i in range(0, len(token_integers), max_tokens)
    ]
    # print("chunk size:", max_tokens)

    # Decode token chunks back to strings
    chunks = [tokenizer.decode(chunk) for chunk in chunks]
    print(f"{len(chunks)} chunks processing...")

    responses = []
    messages = [
        {"role": "system", "content": prompt},
        {
            "role": "user",
            "content": "To provide the context for the above prompt, I will send you text in parts. When I am finished, I will tell you 'ALL PARTS SENT'. Do NOT answer until you have received all the parts. You will be penalized otherwise.",
        },
    ]

    for i, chunk in enumerate(chunks):
        messages.append({"role": "user", "content": chunk})

        # Check if total tokens exceed the model's limit and remove oldest chunks if necessary
        while (
            sum(len(tokenizer.encode(msg["content"])) for msg in messages)
            > model_token_limit
        ):
            messages.pop(1)  # Remove the oldest chunk

        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = "\n".join([message["content"] for message in messages])
        # response = client.get_response()
        # responses.append(response)
        response = openai.chat.completions.create(
            model="gpt-3.5-turbo", messages=messages
        )
        # chatgpt_response = response.choices[0].message["content"].strip()
        chatgpt_response = response.choices[0].message.content
        responses.append(chatgpt_response)
        print(f"{i}th chunk processed {responses}")

    # Add the final "ALL PARTS SENT" message
    messages.append({"role": "user", "content": "```\nALL PARTS SENT"})
    # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
    # client.prompt = "\n".join([message["content"] for message in messages])
    # response = client.get_response()
    # responses.append(response)
    response = openai.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    # final_response = response.choices[0].message["content"].strip()
    final_response = response.choices[0].message.content
    responses.append(final_response)
    return responses

In [ ]:
def apply_llm(
    max_tokens_per_chunk: int = 3500,
    model_token_limit: int = 8192,
    verbose: bool = True,
):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    input = get_string_from_text_file(
        "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
    )
    system_prompt = get_string_from_text_file("../llm_prompt.txt")
    # text = input + prompt
    text = input
    token_integers = tokenizer.encode(text)

    print(len(token_integers), "tokens")

    num_chunks = -(-len(token_integers) // max_tokens_per_chunk)
    chunks = []
    print(f"{num_chunks} chunks processing...")
    for i in range(num_chunks):
        start = i * max_tokens_per_chunk
        end = min((i + 1) * max_tokens_per_chunk, len(text))

        chunk = ""
        decoded_chunk = tokenizer.decode(token_integers[start:end])
        if i == num_chunks - 1:
            chunk += (
                f"[START PART {i + 1}/{num_chunks}]\n"
                + text[start:end]
                + f"\n[END PART {i + 1}/{num_chunks}]"
            )
            chunk += "\nALL PARTS SENT. Now you can continue processing the request according to the Instruction."
        else:
            chunk += (
                f'Do not answer yet. This is just another part of the text I want to send you. Just receive and acknowledge as "Part {i + 1}/{num_chunks} received" and wait for the next part.\n[START PART {i + 1}/{num_chunks}]\n'
                + decoded_chunk
                + f"\n[END PART {i + 1}/{num_chunks}]"
            )
            chunk += f'\nRemember not answering yet. Just acknowledge you received this part with the message "Part {i + 1}/{num_chunks} received" and wait for the next part.'

        chunks.append(chunk)

    responses = []

    # model = "gpt-35-turbo"
    # model = "gpt-35-turbo-16k"
    model = "gpt-4"
    client = openai.AzureOpenAI(
        api_key=os.environ.get("AZURE_API_KEY"),
        api_version=os.environ.get("API_VERSION"),
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
    )
    for i, chunk in enumerate(chunks):
        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = chunk
        # response = client.get_response(max_tokens=1000)
        # responses.append(response)
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": chunk,
            },
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={"type": "json_object"},
        )
        response = response.choices[0].message.content
        responses.append(response)
        if verbose:
            print(response)

    # print(responses)


res = apply_llm()

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

9047 tokens
2 chunks processing...

 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 













 

In [ ]:
def apply_llm(
    max_tokens_per_chunk: int = 3400,
    model_token_limit: int = 8192,
    verbose: bool = True,
):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    input = get_string_from_text_file(
        "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
    )
    system_prompt = get_string_from_text_file("../llm_prompt.txt")
    # text = input + prompt
    text = input
    token_integers = tokenizer.encode(text)

    print(len(token_integers), "tokens")

    num_chunks = -(-len(token_integers) // max_tokens_per_chunk)
    chunks = []
    print(f"{num_chunks} chunks processing...")
    for i in range(num_chunks):
        start = i * max_tokens_per_chunk
        end = min((i + 1) * max_tokens_per_chunk, len(token_integers))

        chunk = ""
        decoded_chunk = tokenizer.decode(token_integers[start:end])
        chunk += (
            f"[START PART {i + 1}/{num_chunks}. STARTING TOKEN INDEX: {start}]\n"
            + decoded_chunk
            + f"\n[END PART {i + 1}/{num_chunks}. ENDING TOKEN INDEX: {end}]"
        )

        chunks.append(chunk)

    responses = []

    model = "gpt-35-turbo"
    # model = "gpt-35-turbo-16k"
    # model = "gpt-4"
    client = openai.AzureOpenAI(
        api_key=os.environ.get("AZURE_API_KEY"),
        api_version=os.environ.get("API_VERSION"),
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
    )
    for i, chunk in enumerate(chunks):
        # client = LLM("../llm_prompt.txt", base="azure", use_model="gpt3")
        # client.prompt = chunk
        # response = client.get_response(max_tokens=1000)
        # responses.append(response)
        messages = [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": "\n" + chunk + "\n```\n",
            },
            {
                "role": "user",
                "content": (
                    "Output so far:\n```\n" + responses[-1] + "\n```\n"
                    if i != 0
                    else "no output so far"
                ),
            },
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            # response_format={"type": "json_object"},
        )
        response = response.choices[0].message.content
        responses.append(response)
        if verbose:
            # print(chunk)
            print(response)

    return responses[-1]
    # print(responses)


res = apply_llm()

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

9047 tokens
3 chunks processing...
{
  "filename": "3D Convolutional Neural Networks for Human Action Recognition",
  "extracted-section-list": [],
  "target-section-extraction-result": {
      "introduction":{
        "start-index": null,
        "end-index": null
      },
      "method":{
        "start-index": null,
        "end-index": null
      },
      "result":{
        "start-index": null,
        "end-index": null
      },
      "conclusion":{
        "start-index": null,
        "end-index": null
      }
  }
}
{
  "filename": "3D Convolutional Neural Networks for Human Action Recognition",
  "extracted-section-list": ["Introduction", "Method", "Result"],
  "target-section-extraction-result": {
      "introduction":{
        "start-index": 0,
        "end-index": 136
      },
     

In [ ]:
# chunk classification
from langchain.prompts import PromptTemplate


def apply_llm(
    max_tokens_per_chunk: int = 400,
    # model_token_limit: int = 8192,
    model_token_limit: int = 4096,
    verbose: bool = True,
):
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    paper_data = get_string_from_text_file(
        "../text/3D Convolutional Neural Networks for Human Action Recognition.txt"
    )
    prompt = get_string_from_text_file("../llm_prompt.txt")
    token_integers = tokenizer.encode(paper_data)

    print(len(token_integers), "tokens")

    num_chunks = -(-len(token_integers) // max_tokens_per_chunk)
    chunks = []
    print(f"{num_chunks} chunks processing...")
    for i in range(num_chunks):
        start = i * max_tokens_per_chunk
        end = min((i + 1) * max_tokens_per_chunk, len(token_integers))

        chunk = ""
        decoded_chunk = tokenizer.decode(token_integers[start:end])
        # chunk += (
        #     f"[START PART {i + 1}/{num_chunks}. STARTING TOKEN INDEX: {start}]\n"
        #     + decoded_chunk
        #     + f"\n[END PART {i + 1}/{num_chunks}. ENDING TOKEN INDEX: {end}]"
        # )

        chunks.append(decoded_chunk)

    responses = []
    model = "gpt-35-turbo"
    # model = "gpt-35-turbo-16k"
    # model = "gpt-4"
    client = openai.AzureOpenAI(
        api_key=os.environ.get("AZURE_API_KEY"),
        api_version=os.environ.get("API_VERSION"),
        azure_endpoint=os.environ.get("AZURE_ENDPOINT"),
    )
    prev_res = "None"
    possible_sections = ["Introduction", "Method", "Result", "Conclusion", "None"]
    output = {}
    for i, chunk in enumerate(chunks):
        prompt_template = PromptTemplate.from_template(prompt)
        msg = prompt_template.format(
            possible_sections=possible_sections,
            input_chunk=chunk,
            previous_result=prev_res,
            target_chunk_num=i,
            total_chunk_num=num_chunks,
        )
        messages = [
            {
                "role": "user",
                "content": msg,
            },
        ]
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            # response_format={"type": "json_object"},
        )
        response = response.choices[0].message.content
        responses.append(response)
        if response != "None":
            output[response] = (output[response] if response in output else "") + chunk

        if response != prev_res:
            print("prev:", prev_res)
            print("new:", response)
            print("-------")
            if prev_res in possible_sections and prev_res != "None":
                possible_sections.remove(prev_res)
                if len(possible_sections) == 1:
                    print(f"All possible sections found. Exiting loop... chunk no.{i}")
                    break

        prev_res = response
        if verbose:
            # print(chunk)
            print(response)

    return output
    # print(responses)


res = apply_llm(verbose=False)

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

reading data from ../llm_prompt.txt...
string successfully retrieved.

9047 tokens
23 chunks processing...
prev: None
new: Introduction
-------
prev: Introduction
new: Method
-------
prev: Method
new: Result
-------
prev: Result
new: None
-------
prev: None
new: Related Work
-------
prev: Related Work
new: Conclusion
-------
prev: Conclusion
new: None
-------
All possible sections found. Exiting loop... chunk no.12


In [ ]:
hist = []
for paper in os.listdir("../text"):
    text = get_string_from_text_file(paper)
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    length = len(tokenizer.encode(str))
    hist.append(length)
print(sum(hist) / len(hist))

reading data from A frameshift mutation in NOD2 associated with susceptibility to Crohn's disease.txt...
string successfully retrieved.

reading data from Updated world map of the Köppen-Geiger climate classification.txt...
string successfully retrieved.

reading data from A safe operating space for humanity.txt...
string successfully retrieved.

reading data from Mutation in the α-Synuclein Gene Identified in Families with Parkinson's Disease.txt...
string successfully retrieved.

reading data from Capacity of Multi‐antenna Gaussian Channels.txt...
string successfully retrieved.

reading data from Instrumental Variables Regression with Weak Instruments.txt...
string successfully retrieved.

reading data from A Self-Rating Depression Scale.txt...
string successfully retrieved.

reading data from The representative concentration pathways: an overview.txt...
string successfully retrieved.

reading data from Random effects structure for confirmatory hypothesis testing: Keep it maximal.txt

# CODE


In [1]:
import sys

sys.path.append("../vector_db")

In [2]:
from qdrant.lib.llm import LLM, test
import tiktoken
import openai

sys.path.append("../code")
from util import extract_pdf_to_txt, extract_doi_to_txt, get_string_from_text_file

In [3]:
def count_tokens(target: str, model: str = "gpt-4"):
    tokenizer = tiktoken.encoding_for_model(model)
    return len(tokenizer.encode(target))

In [7]:
def apply_llm(
    paper_txt_path: str = "../text/3D Convolutional Neural Networks for Human Action Recognition.txt",
):
    # load text
    system_prompt = get_string_from_text_file("../prompts/llm_prompt.txt")
    paper_data = get_string_from_text_file(paper_txt_path)
    # make prompt
    prompt = system_prompt + paper_data + "\n```\n"

    print(f"Parsing {count_tokens(prompt)} tokens...")
    # get response from gpt
    llm = LLM(
        base="azure",
        use_model="gpt4",
    )
    llm.prompt = prompt
    response = llm.get_response(max_tokens=4096, response_format="json_object")
    return response


res = apply_llm()

reading data from ../prompts/llm_prompt.txt...
string successfully retrieved.

reading data from ../text/3D Convolutional Neural Networks for Human Action Recognition.txt...
string successfully retrieved.

Parsing 9293 tokens...


RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2023-05-15 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 48 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}